In [29]:
from typing import List


def compute_probability(time: int, time_distribution: List[int]) -> float:
    """
    Compute the probability of a train departing or arriving at a given time
    given a list of departure times or arrival times
    """
    # count the number of trains departing/arriving at the given time
    count = time_distribution.count(time)
    # count the total number of trains
    total = len(time_distribution)
    # compute the probability
    probability = count / total
    return probability

def compute_probability_to_arrive_at_or_before(time_limit: int, arrival_times: List[int]) -> float:
    """
    Compute the probability of arriving at or before a given time
    given a list of arrival times
    """
    # get first (minimum) arrival time
    min_arrival_time = arrival_times[0] # note: should be the first, we assume we get a sorted list
    # get last (maximum) arrival time -> either the latest arrival time or the time limit if that is earlier
    max_arrival_time = min(arrival_times[-1], time_limit)
    # for each arrival time, compute the probability of arriving at that time (which is the probability of arriving at or before the time limit)
    probabilities = [
        compute_probability(time, arrival_times)
        for time in range(min_arrival_time, max_arrival_time + 1)
    ]
    # sum the probabilities
    probability = sum(probabilities)
    return probability

In [30]:
# @todo -> this is a simplification, we need to find a data structure that allows us to store the stations and their connections
# @todo -> here we only have one "destination" per (unique) station, but in reality, we can have multiple connections -> but only one for a unique trip!
stations = {
    'Zurich': {
        'arrival_times': [0, 0, 0],
        'departure_times': [5, 6, 10],
        'to': 'Olten'
    },
    'Olten': {
        'arrival_times': [15, 15, 20],
        'departure_times': [20, 20, 25],
        'to': 'Bern'
    },
    'Bern': {
        'arrival_times': [30, 32, 35],
        'departure_times': [35, 37, 40],
        'to': 'Brig'
    },
    'Brig': {
        'arrival_times': [45, 47, 50],
        'departure_times': [55, 57, 60], # not important for this example
        'to': 'Milan' # not important for this example
    },
}


In [31]:
# first, create unique lists of the departure and arrival times for each station
# @todo -> question: we should not sort it, because the order means which departure and arrival times belong together?
# @todo -> a unique list here is also wrong -> otherwise the probabilities are wrong
# for station in stations.values():
#     station['arrival_times'] = list(set(station['arrival_times']))
#     station['departure_times'] = list(set(station['departure_times']))


# add the departure probabilities -> creating a tuple from the departure time and the probability
for station in stations.values():
    station['departure_probabilities'] = [
        (time, compute_probability(time, station['departure_times']))
        for time in station['departure_times']
    ]

stations

{'Zurich': {'arrival_times': [0, 0, 0],
  'departure_times': [5, 6, 10],
  'to': 'Olten',
  'departure_probabilities': [(5, 0.3333333333333333),
   (6, 0.3333333333333333),
   (10, 0.3333333333333333)]},
 'Olten': {'arrival_times': [15, 15, 20],
  'departure_times': [20, 20, 25],
  'to': 'Bern',
  'departure_probabilities': [(20, 0.6666666666666666),
   (20, 0.6666666666666666),
   (25, 0.3333333333333333)]},
 'Bern': {'arrival_times': [30, 32, 35],
  'departure_times': [35, 37, 40],
  'to': 'Brig',
  'departure_probabilities': [(35, 0.3333333333333333),
   (37, 0.3333333333333333),
   (40, 0.3333333333333333)]},
 'Brig': {'arrival_times': [45, 47, 50],
  'departure_times': [55, 57, 60],
  'to': 'Milan',
  'departure_probabilities': [(55, 0.3333333333333333),
   (57, 0.3333333333333333),
   (60, 0.3333333333333333)]}}

In [34]:
# for simplicity -> first step, compute probability that we arrive at Olten with a start time 0, budget of 17 and transfer time of 5 minutes (as the default transfer time)
# this is only an intermediate step to understand the reliability computation better

# @todo -> this is a simplification, because I know that I go from Zurich to Olten, but in reality, I need to find the connection that goes from Zurich to Olten
compute_probability_to_arrive_at_or_before(15, stations['Olten']['arrival_times'])
compute_probability_to_arrive_at_or_before(20, stations['Olten']['arrival_times'])

# add the arrival probabilities (in the same order as the arrival times) to the stations
for station in stations.values():
    # @todo / note: this only is correct between the first and second station, because we need to consider making the other connections for further stations
    station['arrival_probabilities'] = [
        (time, compute_probability_to_arrive_at_or_before(time, station['arrival_times']))
        for time in station['arrival_times']
    ]
stations

{'Zurich': {'arrival_times': [0, 0, 0],
  'departure_times': [5, 6, 10],
  'to': 'Olten',
  'departure_probabilities': [(5, 0.3333333333333333),
   (6, 0.3333333333333333),
   (10, 0.3333333333333333)],
  'arrival_probabilities': [(0, 1.0), (0, 1.0), (0, 1.0)]},
 'Olten': {'arrival_times': [15, 15, 20],
  'departure_times': [20, 20, 25],
  'to': 'Bern',
  'departure_probabilities': [(20, 0.6666666666666666),
   (20, 0.6666666666666666),
   (25, 0.3333333333333333)],
  'arrival_probabilities': [(15, 0.6666666666666666),
   (15, 0.6666666666666666),
   (20, 1.0)]},
 'Bern': {'arrival_times': [30, 32, 35],
  'departure_times': [35, 37, 40],
  'to': 'Brig',
  'departure_probabilities': [(35, 0.3333333333333333),
   (37, 0.3333333333333333),
   (40, 0.3333333333333333)],
  'arrival_probabilities': [(30, 0.3333333333333333),
   (32, 0.6666666666666666),
   (35, 1.0)]},
 'Brig': {'arrival_times': [45, 47, 50],
  'departure_times': [55, 57, 60],
  'to': 'Milan',
  'departure_probabilities': [(

In [ ]:
# Now - also for simplicity, compute the probability of making a connection (specifically, from Zurich to Olten)

In [54]:
def compute_connection_probability(departure_time_probabilities: List[int], arrival_time_probabilities: List[float], stations: List, station: str, transfer_time=5) -> float:
    # @todo -> refactor a bit (to make it useful for the next steps, e.g. how the parameters are passed and how the data is stored)
    """
    Compute the probability of making a connection given a list of departure times and a list of arrival time probabilities
    """
    # for each departure time, compute the probability of making a connection
    probabilities = []
    print("departure_time_probabilities")
    print(departure_time_probabilities)
    print("arrival_time_probabilities")
    print(arrival_time_probabilities)
    # multiply probability of departing at time t with the probability of arriving at or before time (t - transfer time)
    for i in range(len(departure_time_probabilities)):
        # only consider the arrival time probabilities that are before the departure time
        arrival_probability = compute_probability_to_arrive_at_or_before(departure_time_probabilities[i][0] - transfer_time, stations[station]['arrival_times'])
        probabilities.append(departure_time_probabilities[i][1] * arrival_probability)
    print(probabilities)
    # sum the probabilities
    probability = sum(probabilities)
    return probability

# we assume that we already have unique lists of tuples representing the departure and arrival times and their probabilities
# sort the lists by time
departure_probabilities = list(set(stations['Olten']['departure_probabilities']))
departure_probabilities.sort(key=lambda x: x[0])
arrival_probabilities = list(set(stations['Olten']['arrival_probabilities']))
arrival_probabilities.sort(key=lambda x: x[0])
compute_connection_probability(departure_probabilities, arrival_probabilities, stations, 'Olten')

departure_time_probabilities
[(20, 0.6666666666666666), (25, 0.3333333333333333)]
arrival_time_probabilities
[(15, 0.6666666666666666), (20, 1.0)]
[0.4444444444444444, 0.3333333333333333]


0.7777777777777777

In [ ]:
#